In [24]:
import numpy as np
import pandas as pd
from sklearn import datasets

data=datasets.load_iris()

In [25]:
X=data.data[:]
y=data.target

In [78]:
class Node():
    def __init__(self,feature=None,threshold=None,left=None,right=None,gain=None,value=None):
        #constructor to create a node 
        
        #if not a leaf node:
        self.feature_index=feature
        self.threshold=threshold
        self.left=left
        self.right=right
        self.info_gain=gain
        
        #if it is a leaf node, above values are left null
        self.value=value

In [79]:
class DecisionTree():
    
    def __init__(self,min_samples=2,max_depth=2):
        self.min_samples=min_samples #if there is only min samples number of points, no need to divide further
        self.max_depth=max_depth #pruning the tree
    
    def build_tree(self,dataset, current_depth=0):
        X=dataset[:,:-1]
        y=dataset[:,-1]
        n_samples, n_features = X.shape #rows and columns
        if n_samples>self.min_samples and current_depth<self.max_depth:#stopping conditions
            best_split=self.get_best_split(dataset,n_samples,n_features)
            
            if (best_split["info_gain"]>0): # if= 0 , pure class, no need to split further
                left_node=self.build_tree(best_split['left_dataset'],current_depth+1)
                right_node=self.build_tree(best_split['right_dataset'],current_depth+1)
            
                return Node(best_split["feature_index"],best_split["threshold"],left_node,right_node,best_split["info_gain"])#create a new node with the feature  index, threshold,info gain and the left and right subtrees attached to it 
        
        
        #if it is a leaf node
        
        leaf_value=self.calculate_leaf_value(y)
        return Node(value=leaf_value) #create a leaf node with just the class labels
    
    def get_best_split(self,dataset,num_samples,num_features):
        best_split={} #dictionary to return the required fields
        max_info_gain=-float('inf')
        
        for feature_index in range(num_features): #in each column
            feature_values=dataset[:,feature_index] #selecting the column
            possible_thresholds=np.unique(feature_values) #possible unique values that can be used as a threshold
            
            for threshold in possible_thresholds: #for each threshold
                left_set, right_set=self.split(dataset, feature_index,threshold) #get the less than and greater than rows from the column
                
                if len(left_set)>0 and len(right_set>0): #if both left and right exist, compute information gain on that split
                    y,left_y,right_y=dataset[:,-1],left_set[:,-1],right_set[:,-1]
                    
                    curr_info_gain=self.information_gain(y,left_y,right_y)
                    
                    if curr_info_gain>max_info_gain: #if it is greater than the current max info gain, update the dictionary
                        best_split["feature_index"]=feature_index
                        best_split["threshold"]=threshold
                        best_split["left_dataset"]=left_set
                        best_split["right_dataset"]=right_set
                        best_split["info_gain"]=curr_info_gain
                        max_info_gain=curr_info_gain
                        
                        #repeat the process for all thresholds and all columns
        
        return best_split
    
    
    def split(self,dataset,feature_index,threshold): #getting less than and greater than rows
        
        dataset_left=np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right=np.array([row for row in dataset if row[feature_index]>threshold])
        
        return dataset_left,dataset_right
    
    def information_gain(self,parent,l_child,r_child):
        
        weight_l=len(l_child)/len(parent)
        weight_r=len(r_child)/len(parent)
        
        gain=self.gini_index(parent)-(weight_l*self.gini_index(l_child)+weight_r*self.gini_index(r_child))
        
        return gain
    
    def gini_index(self,y):
        
        class_labels=np.unique(y)
        gini=0
        for cls in class_labels:
            p_cls=len(y[y==cls])/len(y)
            gini+=p_cls**2
        gini=1-gini
        return gini
    
    def calculate_leaf_value(self,Y): #when info gain becomes zero/depth reaches max, assign label of max number of y values to that leaf node
        
        Y=list(Y)
        
        return max(Y,key=Y.count)
        
        
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

            

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [80]:
model=DecisionTree(min_samples=3,max_depth=3)

In [81]:
model.fit(X_train,y_train.reshape(-1,1))

In [82]:
model.print_tree()

X_2 <= 1.9 ? 0.32087246376811596
 left:0.0
 right:X_3 <= 1.7 ? 0.3681068842808487
  left:X_2 <= 5.1 ? 0.08895044629116648
    left:1.0
    right:2.0
  right:X_2 <= 4.8 ? 0.01942421089143239
    left:2.0
    right:2.0


In [83]:
y_pred=model.predict(X_test)

In [86]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.94      1.00      0.97        15
           2       1.00      0.94      0.97        16

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

